In [1]:
import string
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Enzo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Consigna 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

### Carga de datos y vectorización

In [2]:
# Carga del dataset 20newsgroups.
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Vectorización con TF-IDF del set de datos de entrenamiento.
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data) # Matriz documento-término
y_train = newsgroups_train.target

docs_count = X_train.shape[0]
print(f'El corpus consta de {docs_count} documentos\n')

print(f'Las clases en las que los documentos son clasificados son\n{newsgroups_test.target_names}\n')

# Porcentaje de documentos para cada clase.
for t in np.unique(newsgroups_test.target):
    print(f'El {np.count_nonzero(newsgroups_test.target == t) / docs_count:.2%} de los datos pertenece a la clase {newsgroups_test.target_names[t]}')

print('\nAlgunos de los términos del vectorizador:')
for term in np.sort(list(tfidfvect.vocabulary_.keys()))[20000:20020]:
    print(term)

El corpus consta de 11314 documentos

Las clases en las que los documentos son clasificados son
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

El 2.82% de los datos pertenece a la clase alt.atheism
El 3.44% de los datos pertenece a la clase comp.graphics
El 3.48% de los datos pertenece a la clase comp.os.ms-windows.misc
El 3.46% de los datos pertenece a la clase comp.sys.ibm.pc.hardware
El 3.40% de los datos pertenece a la clase comp.sys.mac.hardware
El 3.49% de los datos pertenece a la clase comp.windows.x
El 3.45% de los datos pertenece a la clase misc.forsale
El 3.50% de los datos pertenece a la clase rec.autos
El 3.52% de los datos pertenec

### Vectorizar 5 documentos y estudiar similitud

In [3]:
# Seleccionar al azar los índices de 5 documentos procedentes del set de entrenamiento.
# Para cada uno de ellos, obtener los índices de los 5 documentos más similares y almacenar
# en un diccionario.
indices = np.random.randint(0, X_train.shape[0] - 1, size=5)
doc_to_similars = {}
for i in indices:
    cos_sim = cosine_similarity(X_train[i], X_train)[0]
    most_sim = np.argsort(cos_sim)[::-1][1:6]
    doc_to_similars[i] = most_sim
    print(f'Los índices de los 5 documentos más similares al documento en índice {i} son: {most_sim}')

Los índices de los 5 documentos más similares al documento en índice 10419 son: [9276 1533 7324 1292 7542]
Los índices de los 5 documentos más similares al documento en índice 2747 son: [1834 6552 1533 9623 3410]
Los índices de los 5 documentos más similares al documento en índice 8642 son: [7595  949  502 9247 6977]
Los índices de los 5 documentos más similares al documento en índice 3442 son: [8405 8965 6176   77 3819]
Los índices de los 5 documentos más similares al documento en índice 10627 son: [6269 7924 6634 5829 7394]


#### Analizar documento 1

In [4]:
# El documento 1 es:
doc_idx = 0
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')



	You talking bout the same Noemi I know? She makes me think of big bore
hand guns and extreme weirdness. This babe rode a CSR300 across the desert! And
a borrowed XL100 on the Death Ride. Don't fuck with her man, your making a big
mistake.



-- 
Go fast. Take chances.

Su target es rec.motorcycles


In [5]:
# Los 5 más similares al documento 1 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')


And Noemi makes me think of "cuddle", not "KotL".

Su target es rec.motorcycles


*******
*******  This is somewhat long, but pleas read it!!!!!!!!!!!!!!!!!
*******



Boy am i glad you decided to read this. I've got a problem that 
I need as many people's help from as possible.

Before I go in to the details of this, let me go ahead and tell
you that (though it may sound it) this is not one of those boy
meets girl problem...at least not totally like that to me....Anyway...

OK, I am a 19 year old Sophmore at NCSU. About 10 years ago, my family
and I were vacationing at the coast in a cottage we rented. Across the
street, was ths girl who would whistle at me whenever she saw me...
her name in Erin. Well, we became friends that week at the beach and have
been writing each other for about 10 years....there was a period of about
2 years we lost contact..but that was a while ago. 

By the way...Erin lives in Kansas and me in NC.

OK, last year in one of her letters, she says that she is c

#### Analizar documento 2

In [6]:
# El documento 2 es:
doc_idx = 1
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')

I am sorry to once again bother those of you on this newsgroup. 
If you have any suggestions as to where I might find out about the subject
of this letter (the origin of Morphine, ie. who first isolsted it, and why
he/she attempted such an experiment).  Once agian any suggestion would be
appreciated.
	CSH
p.s. My instructer insists that I get 4 rescources from this newsgroup, so
please send me and info you think may be helpful.  Facts that you know,
but don't know what book they're from are ok.
ATTENTION: If you do NOT like seeing letters such as this one on your
newsgroup direct all complaints to my instructor at <bshayler@eis.CalStat.Edu>


--
 "Kilimanjaro is a pretty tricky climb. Most of it's up, until you reach
the very, very top, and then it tends to slope away rather sharply."
					Sir George Head, OBE (JC)

Su target es sci.med


In [7]:
# Los 5 más similares al documento 2 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Does anyone on this newsgroup happen to know WHY morphine was
first isolated from opium?  If you know why, or have an idea for where I
could look to find this info, please mail me.
	CSH
any suggestionas would be greatly appreciated

--
 "Kilimanjaro is a pretty tricky climb. Most of it's up, until you reach
the very, very top, and then it tends to slope away rather sharply."
					Sir George Head, OBE (JC)

Su target es sci.med



If I have a habit that I really want to break, and I am willing to
make whatever sacrifice I need to make to break it, then I do so.
There have been bad habits of mine that I've decided to put forth the
effort to break, and I've done so; there have been other bad habits
that I've decided are not worth the effort to break.  It's my choice,
either way.

I am my own master.  I choose what I want to do.  I weigh the benefits
of my actions against their consequences, and I use my experience to
help me deal with the unexpected, which in turn make me more experienced

#### Analizar documento 3

In [8]:
# El documento 3 es:
doc_idx = 2
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')

NHL RESULTS FOR GAMES PLAYED 4/15/93.

--------------------------------------------------------------------------------
                                  STANDINGS
      PATRICK              ADAMS              NORRIS              SMYTHE
 TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT
 
xPIT  56 21  7 119  xBOS  51 26  7 109  xCHI  47 25 12 106  xVAN  46 29  9 101
yWAS  42 34  7  91  yQUE  47 27 10 104  yDET  47 28  9 103  yCAL  43 30 11  97
yNJ   40 36  7  87  yMON  48 30  6 102  yTOR  44 29 11  99  yLA   39 35 10  88
yNYI  39 37  7  85  yBUF  38 36 10  86  ySTL  37 36 11  85  yWIN  40 37  7  87
 PHL  35 37 11  81   HAR  26 51  6  58   MIN  36 38 10  82   EDM  26 50  8  60
 NYR  34 38 11  79   OTT  10 70  4  24   TB   23 54  7  53   SJ   11 71  2  24

x - Clinched Division Title
y - Clinched Playoff Berth

--------------------------------------------------------------------------------

Minnesota North Stars  (36-38-10)   1   1   1   -   3
Detroit Red Win

In [9]:
# Los 5 más similares al documento 3 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

NHL RESULTS FOR GAMES PLAYED 4/14/93.

--------------------------------------------------------------------------------
                                  STANDINGS
      PATRICK              ADAMS              NORRIS              SMYTHE
 TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT
 
xPIT  56 21  7 119  xBOS  51 26  7 109  xCHI  46 25 12 104  xVAN  45 29  9  99
yWAS  42 34  7  91  yQUE  47 27 10 104  yDET  46 28  9 101  yCAL  42 30 11  95
yNJ   40 36  7  87  yMON  48 30  6 102  yTOR  44 28 11  99  yLA   39 34 10  88
yNYI  39 37  7  85  yBUF  38 35 10  86   STL  36 36 11  83  yWIN  39 37  7  85
 PHL  34 37 11  79   HAR  26 51  6  58   MIN  36 37 10  82   EDM  26 49  8  60
 NYR  34 38 11  79   OTT  10 70  4  24   TB   23 53  7  53   SJ   11 70  2  24

x - Clinched Division Title
y - Clinched Playoff Berth

--------------------------------------------------------------------------------

Boston Bruins    (51-26-7)   2   2   0   -   4
Ottawa Senators  (10-7

#### Analizar documento 4

In [10]:
# El documento 4 es:
doc_idx = 3
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')






I sold my '86 Sprint last April with 95k on it.  I'd driven it since
the previous July, putting 20k miles on it.  The sensor light used to
light up regularly, starting about 5k miles after I bought it.  
My brother and I rebuilt the engine but used all of the original equipment,
so I suppose the sensor could have used replacement.  Performance (hah,
if you could call it that) did not change.  Perhaps emissions increased,
but how much emissions could a CA-registered 3 cylinder engine produce?
That was a neat car, I held the engine block easily in one hand!  Has
anyone ever driven the 'Turbo' variant?  Just curious...

Su target es rec.autos


In [11]:
# Los 5 más similares al documento 4 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')


I have owned my Sprint from the beginning and that sensor light comes on every
30K to let you know it's time for a check up.  When that light goes on, I just
take it to the delaer tell them "It's that time" and they work on it for 4 hou
and you pay $5 for parts and $100 for labor. (Ahh...what they get for labor)
I currently have about 95K on my Sprint and of all the times I took it in for
service, I never say on the payment sheet anything about the replacement of the
Oxygen sensor.  What the heck is an Oxygen sensor?  As far an I know of, I have
never had that thing replaced in my car and the car is purring like a kitten.
Now, I don't have a/c (Mother Nature does that for me :-) ) and that might have
something to do with it, but I still never heard of an Oxygen sensor.  The only
MAJOR service job I have had on my car (besides getting the tires replaced
if you want to call that a service job) was getting the Rotor, Distributor and
Gasket replaced.  And that was all done within the past

#### Analizar documento 5

In [12]:
# El documento 5 es:
doc_idx = 4
print(newsgroups_train.data[indices[doc_idx]])

# Su target es:
target_doc = newsgroups_train.target_names[y_train[indices[doc_idx]]]
print(f'\nSu target es {target_doc}')

I would e-mail this to you, but my mailserver doesn't recognize you or
something.

Anyway, the worst pitcher on the Yanks. If you mean currently on the team,
then I have to go with Scott "I'm a schizophrenic...No, I'm NOT!"
Kamienicki. Sure, occasionally the guy can pitch well for 5 or 6 innings,
but then he starts to go insane. A sure sign that he's losing his stuff
(and his mind) is when he starts to stalk around the mound between batters
and yell at himself.

The worst all-time Yanks pitcher?  Gotta go with Ed "New York? I have to
pitch in [gulp] New York?" Whitson. 'Nuff said!


--I'm outta here like Vladimir!
-Alan Sepinwall XVIII

Su target es rec.sport.baseball


In [13]:
# Los 5 más similares al documento 5 son:
targets_accum = 0
for i in range(5):
    j = doc_to_similars[indices[doc_idx]][i]
    print(newsgroups_train.data[j])
    target_similar = newsgroups_train.target_names[y_train[j]]
    targets_accum += 1 if target_doc == target_similar else 0
    print(f'\nSu target es {target_similar}')
    print('\n====================================================\n')

print(f'\nEl {100 * targets_accum / 5}% dos los 5 más similares comparten el mismo target.')

Melido came off the DL today and will start tonight against the Rangers.
(Now, if only he can go the distance so that the bullpen doesn't have to
come in.....)

--I'm outta here like Vladimir!
-Alan Sepinwall

Su target es rec.sport.baseball




I agree, although I would have no idea how to go about doing it. But
you've got my vote.


--I'm outta here like Vladimir
--Alan

Su target es rec.sport.baseball


Alan Sepinwall writes

George will do the only logical thing he can do when the Yanks' bullpen isn't  
performing -- fire the manager.


Su target es rec.sport.baseball


Accounts of Anti-Armenian Human Right Violations in Azerbaijan #013
                 Prelude to Current Events in Nagorno-Karabakh

   +---------------------------------------------------------------------+
   |                                                                     |
   | I said that on February 27, when those people were streaming down   |
   | our street, they were shouting, "Long live Turkey!" and "

### Comentarios

Se tomaron los 5 documentos más similares a cada uno de 5 documentos elegidos al azar. Se usó como métrica la distancia del coseno y a TF-IDF como técnica de vectorización. Para cada uno de los documentos analizados se calculó el porcentaje de los 5 más similares que poseen el mismo target que el documento analizado. En sucesivas pruebas se observó que a veces dicho porcentaje fue inferior al 50%, lo que sugiere que la técnica de vectorización empleada podría ser mejorada.

## Consigna 2

Entrenar modelos de clasificación Naive Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naive Bayes Multinomial
y ComplementNB.

### Naive Bayes y Complement Naive Bayes usando la vectorización obtenida hasta aquí y parámetros por defecto

In [14]:
# Instanciar y fitear un multinomial Naive Bayes classifier.
mult_nb = MultinomialNB()
mult_nb.fit(X_train, y_train)

# Vectorizar set de datos de testing y realizar predict.
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  mult_nb.predict(X_test)

# Calcular F1-score macro.
print(f'F1 score macro con multinomial Naive Bayes:', f1_score(y_test, y_pred, average='macro'))

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

# Calcular F1-score macro.
print(f'F1 score macro con complement Naive Bayes:', f1_score(y_test, y_pred, average='macro'))

F1 score macro con multinomial Naive Bayes: 0.5854345727938506
F1 score macro con complement Naive Bayes: 0.692953349950875


### Complement Naive Bayes usando stopwords

In [15]:
# Volver a instanciar el vectorizador, esta vez usando stop words del inglés.
tfidfvect = TfidfVectorizer(stop_words='english')
X_train = tfidfvect.fit_transform(newsgroups_train.data)

X_test = tfidfvect.transform(newsgroups_test.data)

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

print(f'F1 score macro con complement Naive Bayes y vectorización con stopwords es :', f1_score(y_test, y_pred, average='macro'))

F1 score macro con complement Naive Bayes y vectorización con stopwords es : 0.6936107849650025


### Complement Naive Bayes usando preprocesamiento con NLTK

In [16]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def nltk_preprocess(document):
    '''
    Aplica tokenization, filtra stopwords en inglés, filtra signos de puntuación y aplica lemmatization
    a un documento recibido como parámetro. Retorna el documento preprocesado.
    '''
    tokens = word_tokenize(document.lower())
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Preprocesar todos los documentos.
newsgroups_train.data = [nltk_preprocess(document) for document in newsgroups_train.data]
newsgroups_test.data = [nltk_preprocess(document) for document in newsgroups_test.data]

print('Ejemplo de cómo se ve un documento luego de haber sido preprocesado: ', newsgroups_train.data[0])

# Volver a instanciar el vectorizador.
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

X_test = tfidfvect.transform(newsgroups_test.data)

# Instanciar y fitear un complement Naive Bayes classifier.
compl_nb = ComplementNB()
compl_nb.fit(X_train, y_train)

# Realizar predict.
y_pred = compl_nb.predict(X_test)

print(f'\nF1 score macro con complement Naive Bayes y preprocesamiento con NLTK:', f1_score(y_test, y_pred, average='macro'))

Ejemplo de cómo se ve un documento luego de haber sido preprocesado:  wondering anyone could enlighten car saw day sport car looked late early called bricklin door really small addition front bumper separate rest body know anyone tellme model name engine spec year production car made history whatever info funky looking car please

F1 score macro con complement Naive Bayes y preprocesamiento con NLTK: 0.6793289730320086


## Consigna 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

### Vectorización de términos

In [17]:
# Transponer X_train para obtener la matriz de términos a documentos. Cada fila puede ser interpretada como una vectorización de un término.
X_train_t = X_train.T

# Contruir diccionario que mapea índices con términos.
idx_to_term = { v: k for k, v in tfidfvect.vocabulary_.items() }

print(f'Se vectorizaron {X_train_t.shape[0]} terminos de {X_train_t.shape[1]} componentes cada uno.')

Se vectorizaron 55745 terminos de 11314 componentes cada uno.


### Vectorizar 5 términos y estudiar similitud

In [18]:
# Seleccionar al azar los índices de 5 términos procedentes del set de entrenamiento.
# Para cada uno de ellos, obtener los índices de los 5 términos más similares y almacenar
# en un diccionario.
indices = np.random.randint(0, X_train_t.shape[0] - 1, size=5)
term_to_similars = {}
for i in indices:
    cos_sim = cosine_similarity(X_train_t[i], X_train_t)[0]
    most_sim = np.argsort(cos_sim)[::-1][1:6]
    term_to_similars[i] = most_sim
    print(f'Los índices de los 5 términos más similares al término en índice {i} son: {most_sim}')

Los índices de los 5 términos más similares al término en índice 32332 son: [24506 47934 34690  5005  5012]
Los índices de los 5 términos más similares al término en índice 7899 son: [40598 26034 15562  8272 53296]
Los índices de los 5 términos más similares al término en índice 43337 son: [ 1577 43337 12632 17166  3759]
Los índices de los 5 términos más similares al término en índice 11404 son: [44250 39665 53277 22705 36843]
Los índices de los 5 términos más similares al término en índice 34170 son: [11916 15438 20732 17490  6158]


#### Analizar término 1

In [19]:
# El término 1 es:
term_idx = 0
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 32332 es nasil


In [20]:
# Los 5 más similares al término 1 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 24506. Term: ittihad


Índice: 47934. Term: telgraflarin


Índice: 34690. Term: osmaniye


Índice: 5005. Term: birakismasi


Índice: 5012. Term: birinci




#### Analizar término 2

In [21]:
# El término 2 es:
term_idx = 1
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 7899 es christie


In [22]:
# Los 5 más similares al término 2 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 40598. Term: reponded


Índice: 26034. Term: kille


Índice: 15562. Term: essene


Índice: 8272. Term: clh


Índice: 53296. Term: william




#### Analizar término 3

In [23]:
# El término 3 es:
term_idx = 2
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 43337 es separet


In [24]:
# Los 5 más similares al término 3 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 1577. Term: amadi


Índice: 43337. Term: separet


Índice: 12632. Term: dinning


Índice: 17166. Term: fireplace


Índice: 3759. Term: backyard




#### Analizar término 4

In [25]:
# El término 4 es:
term_idx = 3
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 11404 es decieved


In [26]:
# Los 5 más similares al término 4 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 44250. Term: sinlessness


Índice: 39665. Term: reamin


Índice: 53277. Term: wile


Índice: 22705. Term: immaclute


Índice: 36843. Term: plenitude




#### Analizar término 5

In [27]:
# El término 5 es:
term_idx = 4
print(f'El término en índice {indices[term_idx]} es {idx_to_term[indices[term_idx]]}')

El término en índice 34170 es olivia


In [28]:
# Los 5 más similares al término 5 son:
for i in range(5):
    j = term_to_similars[indices[term_idx]][i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')
    print('\n====================================================\n')

Índice: 11916. Term: denoument


Índice: 15438. Term: errol


Índice: 20732. Term: havilland


Índice: 17490. Term: flynn


Índice: 6158. Term: buff




### Comentarios

In [29]:
# Ejemplo tomando la palabra "moon".
moon_index = tfidfvect.vocabulary_['moon']
cos_sim = cosine_similarity(X_train_t[moon_index], X_train_t)[0]
most_sim = np.argsort(cos_sim)[::-1][1:6]
term_to_similars = most_sim
print(f'Los índices de los 5 términos más similares a moon {i} son: {most_sim}')

for i in range(5):
    j = term_to_similars[i]
    print(f'Índice: {j}. Term: {idx_to_term[j]}')

Los índices de los 5 términos más similares a moon 4 son: [28319 19717 42532  3197  8824]
Índice: 28319. Term: lunar
Índice: 19717. Term: gravitacional
Índice: 42532. Term: sattellite
Índice: 3197. Term: atraction
Índice: 8824. Term: commericalization


Se tomaron los 5 términos más similares a cada uno de 5 términos elegidos al azar. Se usó como métrica la distancia del coseno y a TF-IDF como técnica de vectorización. En sucesivas pruebas se observó que no necesariamente existe semejanza semántica entre los términos comparados. Esto es esperable dada la técnica de vectorización empleada. TF-IDF no capta el significado semántico de las palabras y no considera el contexto en el que las mismas figuran. Palabras sinónimas podrían tener vectores no cercanos en función de las diferentes frencuencias con las que estas palabras puedan aparecer en un determinado corpus. Distinto habría sido el resultado de haber usado word embeddings sobre un corpus suficientemente rico.